# Sequence to Sequence Chatbot

In this notebook, I first train a  Sequence to Sequence text generation architecture based chatbot on a subset of  torchtext's SQuAD 1.0 dataset which consists of question-answer-pairs. Subsequently, I confront the trained model with both questions it has been exposed to and questions it has not been exposed to demonstrate the way the model works.


In [2]:

from src.data_chatbot import questions_answers, load_df, toTensor, show_lengths, vectorize_questions, vectorize_answers
from src.data_chatbot import prepare_text
from src.models_chatbot import Seq2Seq
from src.vocab_chatbot import Vocab


from src.train_chatbot import train
from src.apply_chatbot import apply_chatbot



### Tests

In [ ]:
!python -m pytest -vv src/tests_chatbot.py

### Inspect raw data

In [ ]:
df_train = load_df()
df_train.head()

### Tokenize data

In [ ]:
questions, answers = questions_answers(5000)
show_lengths(questions, answers)

### Create vocabularies

In [ ]:
vQ = Vocab("Questions")
for sequence in [["<SOS>", "<EOS>"]] + questions:
    for token in sequence:
        vQ.indexWord(token)
vA = Vocab("Answers")
for sequence in [["<SOS>", "<EOS>"]] + answers:
    for token in sequence:
        vA.indexWord(token)
print(f"The source vocabulary contains {len(vQ.word2index)} and the target vocabulary contains {len(vA.word2index)} words.")

### Create vectors

In [ ]:
vectorized_questions = vectorize_questions(questions, vQ)
vectorized_answers = vectorize_answers(answers, vA)
print('Vectorization completed.')


### Create model

In [ ]:
input_size = len(vQ.word2index)
hidden_size = 124
output_size = len(vA.word2index) 

dropout_E=0.0
dropout_D=0.0
teacher_forcing_ratio=0.0


model = Seq2Seq(input_size, hidden_size, output_size)

### Train model

In [ ]:
epochs = 30
batch_size = 124
print_each = 1
lr = 0.01
weight_decay = 0
version = str(hidden_size)
train(epochs, batch_size, print_each, lr, model, version, vectorized_questions, vectorized_answers, vQ, vA)

### Apply model

In [ ]:
import torch
version = 124
model.load_state_dict(torch.load(f"model_{version}.pt", map_location=torch.device('cpu')))
print(f"Loading from checkpoint: 'model_{version}.pt'")


max_count = 0
for answer in answers:
    if len(answer) > max_count:
        max_count = len(answer)

model.eval()
string2stop = 'quit'
print(f"Type {string2stop} to finish the chat.\n")

    
while (True):
    question = input("> ")
    if question.strip() == string2stop:
        break
    
    apply_chatbot(model, vectorize_questions([prepare_text(question)],vQ)[0].view(-1,1), vQ, vA, max_count)